# Visualisation and Interactivity improvements
## Objectives
1. ~Complete write function that takes ODC xarray output and converts it to RGBA colormap whilst accounting for single bands~
2. ~Modify `mk_image_overlay` to also accept RGB(A) values~
3. Integrate reprojection to web mercator in parent function
4. Normalising dependent on input, no data, categories, bitmask, boolean
5. ~Sidecar / ipyleaflet auto update instead of creating new tabs~
6. ~Transparency slider~
7. ~Basemap plotting options~
8. Options for time slices
9. ~Display time of image on basemap~
10. ~Auto zoom based on query bounding box~

In [110]:
import odc.algo
import odc.ui

from datacube import Datacube
from datacube.storage.masking import mask_invalid_data
import ipyleaflet
import datacube
import sys
import xarray as xr
import numpy as np
from typing import Tuple, Optional, List
from sidecar import Sidecar
from ipywidgets import IntSlider, widgets as w
from matplotlib import cm
import matplotlib.pyplot as plt
from IPython.display import Image, display
from matplotlib.colors import Normalize
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../../Scripts')
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map

In [111]:
from matplotlib import cm
from matplotlib.colors import Normalize

def colorize(da, cmap='viridis', vmin=None, vmax=None):
    """
    Convert values in single band xarray to colormap values
    Can accept multiple time steps
    da - xarray.DataArray
    cmap - desired colormapping
    vmin / vmax - values for normalisation
    """
    normalized = Normalize(vmin=vmin, vmax=vmax)(da)
    colormapped = cm.get_cmap(cmap)(normalized, bytes=True)
    colormapped_xr = xr.DataArray(data=colormapped, 
                                  coords = da.coords, dims=da.dims+('band',))
    
    colormapped_xr[...,3] = np.isfinite(da) * 255
    return colormapped_xr

In [112]:
from datacube.utils.rio import set_default_rio_config

# Only run if executing in the cloud, will successfully do nothing on NCI
if 'AWS_ACCESS_KEY_ID' in os.environ:
    set_default_rio_config(aws={'region_name': 'auto'},
                           cloud_defaults=True)

In [113]:
dc = Datacube(app='viz')

In [114]:
# Create sidecar and map widget
sc = Sidecar(title='Map')

# Create map and add it to sidecar
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, center=(-25, 133), zoom=3, scroll_wheel_zoom=True)

# Add Full Screen and Layers Controls
m.add_control(ipyleaflet.FullScreenControl())
m.add_control(ipyleaflet.LayersControl())

# Add the opactity slider
slider = w.FloatSlider(min=0, max=1, value=1,        # Opacity is valid in [0,1] range
                       orientation='vertical',       # Vertical slider is what we want
                       readout=False,                # No need to show exact value
                       layout=w.Layout(width='2em')) # Fine tune display layout: make it thinner
m.add_control(ipyleaflet.WidgetControl(widget=slider))

# Add map to sidecar
with sc:
    display(m)

In [115]:

# Define product and red/green/blue bands in the given product
product = 'ls8_nbart_geomedian_annual'
RGB = ('red', 'green', 'blue')

# Region and time of interest
query = dict(
    lat=(-27.60, -27.665),
    lon=(153.33, 153.425),
    time='2018',
)

dss = dc.find_datasets(product=product, **query)
print(f"Found {len(dss)} datasets")

ds = dc.load(
    product=product,             # dc.load always needs product supplied, this needs to be fixed in `dc.load` code
    datasets=dss,                # Datasets we found earlier
    measurements=RGB,            # Only load red,green,blue bands
    group_by='solar_day',        # Fuse all datasets captured on the same day into one raster plane
    output_crs='EPSG:3857',      # Default projection used by Leaflet and most webmaps
    resolution=(-30, 30),      # 200m pixels (1/20 of the native
    progress_cbk=odc.ui.with_ui_cbk())  # Display load progress
ds

# Load data
#query = {
#    'x': (153.33, 153.425),
#    'y': (-27.60, -27.665),
#    'time': ('2018'),
#    'output_crs': 'EPSG:3857',
#    'resolution': (-30, 30),
#    'group_by': 'solar_day'
#}
#
#ds = dc.load(product = 'ls8_nbart_geomedian_annual', **query)
#
#print(str(ds.time[0].values))

Found 1 datasets


<xarray.Dataset>
Dimensions:  (time: 1, x: 4375, y: 4292)
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01
  * y        (y) float64 -3.119e+06 -3.119e+06 ... -3.248e+06 -3.248e+06
  * x        (x) float64 1.698e+07 1.698e+07 1.698e+07 ... 1.711e+07 1.711e+07
Data variables:
    red      (time, y, x) int16 -999 -999 -999 -999 -999 ... -999 -999 -999 -999
    green    (time, y, x) int16 -999 -999 -999 -999 -999 ... -999 -999 -999 -999
    blue     (time, y, x) int16 -999 -999 -999 -999 -999 ... -999 -999 -999 -999
Attributes:
    crs:      EPSG:3857

In [116]:
ds = mask_invalid_data(ds)
ds

<xarray.Dataset>
Dimensions:  (time: 1, x: 4375, y: 4292)
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01
  * y        (y) float64 -3.119e+06 -3.119e+06 ... -3.248e+06 -3.248e+06
  * x        (x) float64 1.698e+07 1.698e+07 1.698e+07 ... 1.711e+07 1.711e+07
Data variables:
    red      (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    green    (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    blue     (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    crs:      EPSG:3857

In [127]:
# Convert dataarray values to colormapping
cm_da = colorize(ds.red, vmin=500, vmax=3000, cmap='Reds')


/usr/local/lib/python3.6/dist-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [120]:
image = odc.ui.mk_image_overlay(cm_da, layer_name='Image1')

In [121]:
zoom = odc.ui.zoom_from_bbox(ds.geobox.geographic_extent.boundingbox)

In [122]:
# Add layer to map
m.zoom = zoom
# Center map on new image
m.center = (query['lat'][0], query['lon'][0])

# Add the opacity slider to the new image
w.jslink((slider, 'value'),         
         (image, 'opacity') )
m.add_layer(image)

In [123]:
#message1 = w.HTML()
#message1.value = str(ds.time[0].values)

# Popup with a given location on the map:
#popup1 = ipyleaflet.Popup(
#    location=(query['lat'][0], query['lon'][0]),
#    child=message1,
#    close_button=False,
#    auto_close=False,
#    close_on_escape_key=False
#)
#m.add_layer(popup1)



In [124]:
import time

polygons, bbox = odc.ui.dss_to_geojson(dss, bbox=True)

def hover_handler(event=None, feature=None, id=None, properties=None):
    label = w.HBox([w.Label(value="Hover to see the timestamp:")])
    m.add_control(ipyleaflet.WidgetControl(widget=label))
    time.sleep(3)
    m.remove_control(widget=label)

In [125]:
geojson = ipyleaflet.GeoJSON(
    data={'type': 'FeatureCollection',
          'features': polygons},
    style={
        'opacity': 0.3,      # Footprint outline opacity
        'fillOpacity': 0     # Do not fill
    },
    hover_style={'color': 'tomato'},  # Style when hovering over footprint
    name="Footprints"                 # Name of the Layer, used by Layer Control widget
)

geojson.on_hover(hover_handler)

In [126]:
m.add_layer(geojson)

In [64]:
label = w.HBox([w.Label(value="Hover to see the timestamp:")])
m.add_control(ipyleaflet.WidgetControl(widget=label))